# Del 3: Optimizacija kode za velike datasete

Pripravimo datasete:

In [1]:
!tar -xJf data/data_del_06.tar.xz -C ./data/

In [2]:
import pandas as pd
import numpy as np

## CPU Bound Programs

### Bounds vs Limitations

<img alt="I/O bounds" src="images/CPU+and+I_O+bounds.png">

### Primer optimizacije

In [9]:
import numpy as np

# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    total = 2 * np.arcsin(np.sqrt(a)) 
    return total

In [10]:
df = pd.read_csv('data/new_york_hotels.csv')

#### Crude looping over DataFrame rows using indices

In [12]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [13]:
%%timeit
# Run the haversine looping function
df['distance'] = haversine_looping(df)

507 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Looping with iterrows()

In [14]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows(): #optimiziranjo sprehanje po rowih
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

176 ms ± 790 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Looping with apply()

In [15]:
%%timeit

# Timing apply on the Haversine function
# Če ne moreš uporabiti vektorizacije, se izogni for zank in uporabi apply.
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)

63 ms ± 826 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Vectorization with Pandas series

In [16]:
%%timeit 
# Vectorized implementation of Haversine applied on Pandas series
df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

1.92 ms ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


####  Vectorization with NumPy arrays

In [17]:
%%timeit
# Vectorized implementation of Haversine applied on NumPy arrays
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

214 µs ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## I/O Bound Programs

### I/O Bounds

<img src="./images/report_assembly.png">

<img src="./images/report_assembly_bidir.png">

### Profiling an I/O bound task

In [9]:
query = '''
SELECT DISTINCT teamID 
FROM Teams 
INNER JOIN TeamsFranchises ON Teams.franchID == TeamsFranchises.franchID 
WHERE TeamsFranchises.active = 'Y';
'''

In [11]:
import cProfile
import sqlite3

conn = sqlite3.connect("data/lahman2015.sqlite")



In [44]:
import cProfile
import sqlite3

query = "SELECT SUM(HR) FROM Batting WHERE teamId=?"
conn = sqlite3.connect("data/lahman2015.sqlite")
cur = conn.cursor()

def calculate_runs(teams):


### Blocking Tasks

In [18]:
import sqlite3

# Create an in memory database.
memory = sqlite3.connect(':memory:')

# Connect to our disk database.
disk = sqlite3.connect('data/lahman2015.sqlite')




In [20]:
import cProfile
import sqlite3



## Optimizing Python Code with pandas

### Basic Looping

### Select columns and rows efficiently


In [18]:
data = pd.read_csv('data/school.csv')
data.head(3)

,School ID,School Name,Building Code,Street Address,City,State,Zip Code
0,02M260,Clinton School Writers and Artists,M933,425 West 33rd Street,Manhattan,NY,10001
1,06M211,Inwood Early College for Health and Informatio...,M052,650 Academy Street,Manhattan,NY,10002
2,01M539,"New Explorations into Science, Technology and ...",M022,111 Columbia Street,Manhattan,NY,10002


In [20]:
data['City'].value_counts().head(10)

Brooklyn            121
Bronx               118
Manhattan           106
Jamaica              13
Long Island City     12
Staten Island        10
Flushing              8
Astoria               6
Elmhurst              5
Rockaway Park         4
Name: City, dtype: int64

In [23]:
top_cities = data['City'].value_counts().head(5).index.to_list()

In [24]:
top_cities

['Brooklyn', 'Bronx', 'Manhattan', 'Jamaica', 'Long Island City']

In [27]:
#slabo

%%time
data['City'][(data['City']).isin(top_cities) == False] = 'Others'

CPU times: user 2.94 ms, sys: 198 µs, total: 3.14 ms
Wall time: 2.53 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
#podatke izbiraj z loc ali iloc, ker je veliko hitrejše

%%timeit
data.loc[(data['City']).isin(top_cities) == False, "City"] = 'Others'

567 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Uporaba biult-in funkciji

### Joining on indexes is faster than joining on columns

Construct some sample data:

In [30]:
n = 100000

i1 = np.arange(n)
np.random.shuffle(i1)
df1 = pd.DataFrame({'i': i1,
                    'j': np.random.randint(1,1000,n),
                    'k': np.random.randint(1,1000,n)})

i2 = np.arange(n)
np.random.shuffle(i1)
df2 = pd.DataFrame({'i': i2,
                    'm': np.random.randint(1,1000,n),
                    'n': np.random.randint(1,1000,n)})

## PRIMER: Pohitritev pandas kode

### Naloga

### Priprava podatkov

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('data/demand_profile.csv')

In [33]:
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date_time   8760 non-null   object 
 1   energy_kwh  8760 non-null   float64
dtypes: float64(1), object(1)
memory usage: 137.0+ KB


In [35]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [36]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].dtype

dtype('<M8[ns]')

In [37]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


In [40]:
def convert(df,column_name):
    return pd.to_datetime(df[column_name], format = '%d/%m/%y %H:%M')

In [ ]:
%%timeit -r 3 -n 30 #3 repetitions, 10 loops
df_converted['date_time'] = convert(df, 'date_time')

In [42]:
df = pd.read_csv('data/demand_profile.csv')
df_converted = df.copy()

In [43]:
%%timeit -r 3 -n 30 #3 repetitions, 10 loops
df_converted['date_time'] = convert(df, 'date_time')


34.1 ms ± 293 µs per loop (mean ± std. dev. of 3 runs, 30 loops each)


In [44]:
df_converted.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


In [45]:
df_test = df_converted.copy()

In [46]:
df_test['cost_cents'] = df['energy_kwh']*28
df_test.head()

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,16.408
1,2013-01-01 01:00:00,0.580,16.240
2,2013-01-01 02:00:00,0.572,16.016
3,2013-01-01 03:00:00,0.596,16.688
4,2013-01-01 04:00:00,0.592,16.576


### 1) Simple Looping Over Pandas Data

In [50]:
def apply_tariff(kwh, hour):
    """Calculates cost of electricity for given hour."""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

In [55]:
df_rac = df_converted.copy()
df_rac['date_time'] = pd.to_datetime(df['date_time'])

In [60]:
df_rac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date_time   8760 non-null   datetime64[ns]
 1   energy_kwh  8760 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 137.0 KB


In [64]:
df_rac['cost_cents'] = apply_tariff(df_rac['energy_kwh'], (df_rac['date_time'].index.hour))

AttributeError: 'RangeIndex' object has no attribute 'hour'

### 2) Looping with .itertuples() and .iterrows()

### 3) Pandas’ .apply()

### 4) Selecting Data With .isin()

### 5) Pandas’ pd.cut() function

### 6) Using NumPy